In [1]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 146.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [36]:

#load dataset
merged_df=pd.read_csv("/content/merged_df.csv")
merged_df.head(2)


,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities,user_id,clicked,content
0,N37243,finance,finance-real-estate,the 25 most desirable places to live in the us...,check out where u s residents would live if th...,https://assets.msn.com/labs/mind/AABvlID.html,[],"[{""Label"": ""United States"", ""Type"": ""G"", ""Wiki...",unknown,1.0,the 25 most desirable places to live in the us...
1,N25540,finance,finance-saving-investing,take heart millennials investing is within you...,news headlines might lead you to believe that ...,https://assets.msn.com/labs/mind/AAEmGBr.html,[],[],unknown,1.0,take heart millennials investing is within you...


In [37]:
# Fill NaN in content and required columns
merged_df['content'] = merged_df['content'].fillna('')
merged_df['user_id'] = merged_df['user_id'].fillna('unknown')
merged_df['news_id'] = merged_df['news_id'].fillna('unknown')


In [38]:
# Content-Based Recommendation
vectorizer=TfidfVectorizer(stop_words='english',max_df=0.8,min_df=5,ngram_range=(1,2))
tfidf_matrix = vectorizer.fit_transform(merged_df['content'])

In [39]:
# make a function content-based recommendation system
def content_base_rec(title, top_n=5):
  title_clean=re.sub(r'\W+', ' ', title).lower().strip()
  title_vec=vectorizer.transform([title_clean])
  sim_scores=cosine_similarity(title_vec, tfidf_matrix).flatten()
  top_indices=sim_scores.argsort()[::-1][:top_n]
  return merged_df.loc[top_indices,['news_id','title','category','subcategory','url','abstract']]

In [40]:
#Collaborative-Based-Recommendation
user_encoder=LabelEncoder()
item_encoder=LabelEncoder()
merged_df['user_enc']=user_encoder.fit_transform(merged_df['user_id'])
merged_df['news_enc']=item_encoder.fit_transform(merged_df['news_id'])

In [41]:
#Create user-item matrics(rows=user,cols=news,values=clicked)
user_item_matrix=merged_df.pivot_table(index='user_enc',columns='news_enc',values='clicked',fill_value=0)

In [42]:
#Compute cosine similarity between users
user_similarity=cosine_similarity(user_item_matrix)
user_similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [43]:
#make a function collaborative recommendation system
def collaborative_base_rec(input_user,df,top_k=5):

  #Encode user_id to index
  user_idx=user_encoder.transform(input_user)

  #Get similarity scores for this user with others
  sim_scores=user_similarity[user_idx]

  #Get clicked items by this user
  user_clicks=user_item_matrix.iloc[user_idx]

  #Calculate weighted sum of clicks by similarity scores
  weighted_scores=sim_scores@user_item_matrix.values

  #remove already clicked items from recommendations
  weighted_scores[user_clicks==1]=0

  #Get top news indices based on scrores
  top_news_indices=weighted_scores.argsort()[::-1][:top_k]

  #Decode back to news_id
  recommended_news_ids=item_encoder.inverse_transform(top_news_indices)

  #Filter original df for those news and drop duplicates
  recommended_news=df[df['news_id'].isin(recommended_news_ids)][['news_id','title','category','subcategory','url','abstract']].drop_duplicates(subset='news_id')
  return recommended_news.reset_index(drop=True)

  collaborative_base_rec(
      input_user='U91836',
      df=merged_df,

  )
  recommendations



In [44]:
#Hybrid Recommendations
def hybrid_recommendations(user_id,title,df,top_n=5):
  #clean title
  title_clean=re.sub(r'\W+', ' ', title).lower().strip()

  #Get content base recommendation
  content_rec=content_base_rec(title_clean)

  #Get Collaborative base recommendation
  collab_rec=collaborative_base_rec(user_id,df)

  #Merge the recommendations
  combine_rec=pd.concat([content_rec,collab_rec]).drop_duplicates().reset_index(drop=True)

  #limit the numer of recommendation to the top 'n'
  combine_rec=combine_rec.head(top_n)
  return combine_rec

  hybrid_recommendations(
      user_id='U91836',
      title="Top investment strategies 2025",
      df=merged_df
  )

In [54]:
!pip install streamlit

In [56]:
!pip install streamlit localtunnel


ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [55]:
# STREAMLIT UI
# ==========================
st.title("🔥 News Recommendation System (Hybrid)")

user_id = st.text_input("Enter User ID:", "U91836")
title = st.text_input("Enter News Title:", "Top investment strategies")

if st.button("Get Recommendations"):
    result = hybrid_recommendations(user_id, title, merged_df)
    st.write("### Recommendations:")
    st.dataframe(result)

2025-11-24 10:48:25.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 10:48:25.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar